In [1]:
from datasets import list_datasets, load_dataset
import numpy as np
import os
os.chdir("/Users/zyxu/Documents/py/NLP/metaLM")

from transformers import DataProcessor, InputExample

import torch
import torch.nn as nn

from src import metaTestDataset, load_taskdata

/Users/zyxu/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# dataset = load_taskdata(task_name='mnli', split = 'train')
# dataset

In [3]:
dataset = metaTestDataset(task_name='mnli', split = 'train')

Found cached dataset glue (/Users/zyxu/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [4]:
dataset.task_name

'mnli'

In [ ]:
sq_idx, label_idx, result = dataset[0]

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
print(sq_idx)
print(label_idx)
print(result.keys())

tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [7]:
result['token_type_ids']

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [8]:
len(result['input_ids'])

30

In [9]:
(result['input_ids'])

tensor([[  101,  1109,  2590,  ...,     0,     0,     0],
        [  101,  1706, 12647,  ...,     0,     0,     0],
        [  101,  1370,  1859,  ...,     0,     0,     0],
        ...,
        [  101,  1135,  1110,  ...,     0,     0,     0],
        [  101,   178,  3983,  ...,     0,     0,     0],
        [  101,  1128,  1221,  ...,     0,     0,     0]])

## dataloader

In [4]:
from torch.utils.data import DataLoader

In [5]:
loader = DataLoader(dataset, batch_size=1)

In [6]:
sq_idx, label_idx, batch = next(iter(loader))

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
batch.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [17]:
batch['attention_mask'].shape

torch.Size([30, 128])

In [18]:
batch['token_type_ids'].shape

torch.Size([30, 128])

In [19]:
batch['input_ids'].shape

torch.Size([30, 128])

In [7]:
for key, val in batch.items():
    batch[key] = val[0]
sq_idx = sq_idx.view(-1)
label_idx = label_idx.view(-1)

### model forawrd

##### encoder

In [8]:
from src import BertEncoder

In [9]:
enc = BertEncoder.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertEncoder: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
batch.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [22]:
batch

{'input_ids': tensor([[  101,  9294,  1122,  ...,     0,     0,     0],
         [  101,  1124,  1108,  ...,     0,     0,     0],
         [  101,  2189, 12138,  ...,     0,     0,     0],
         ...,
         [  101, 20164,  2176,  ...,     0,     0,     0],
         [  101,  5893,  1442,  ...,     0,     0,     0],
         [  101,  1109,  2529,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [10]:
out = enc(**batch)

In [11]:
out.shape

torch.Size([30, 768])

In [12]:
print(sq_idx, "\n",label_idx)

tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1]) 
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])


In [53]:
query = out[sq_idx.view(-1) == 1,:]
query.shape

torch.Size([24, 768])

In [56]:
shot = out[sq_idx.view(-1) == 0,:].view(3,2,-1)
shot.shape

torch.Size([3, 2, 768])

In [62]:
shot.mean(dim = 1).shape

torch.Size([3, 768])

In [68]:
label_idx.view(-1)[sq_idx.view(-1) == 0]

tensor([0, 0, 1, 1, 2, 2])

In [69]:
label_idx.view(-1).unique().shape[0]

3

In [20]:
# a = torch.arange(24).view(6,4)
# print(a, "\n")
# b = a.view(2,3,-1)
# print(b, "\n")
# b.flatten(0,-2)

#### classifier

In [13]:
from src import FSCentroidClassifier

In [14]:
clf = FSCentroidClassifier()

In [15]:
sq_idx

tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1])

In [16]:
num_classes = label_idx.unique().shape[0]
query = out[sq_idx == 1,:] # torch.Size([24, 768])  [Y * Q, D]
shot = out[sq_idx == 0,:].view(num_classes,2,-1)  # torch.Size([3, 2, 768]) [Y, S, D]

In [17]:
clf(shot, query)

tensor([[0.9989, 0.9953, 0.9700],
        [0.9995, 0.9949, 0.9702],
        [0.9918, 0.9990, 0.9851],
        [0.9989, 0.9961, 0.9719],
        [0.9960, 0.9976, 0.9778],
        [0.9999, 0.9932, 0.9682],
        [0.9999, 0.9933, 0.9682],
        [0.9967, 0.9981, 0.9764],
        [0.9947, 0.9974, 0.9762],
        [0.9120, 0.9426, 0.9823],
        [0.9992, 0.9954, 0.9708],
        [0.9999, 0.9938, 0.9689],
        [0.3565, 0.4209, 0.5570],
        [0.9996, 0.9945, 0.9696],
        [0.8929, 0.9203, 0.9651],
        [0.9981, 0.9969, 0.9731],
        [0.9998, 0.9935, 0.9684],
        [0.9999, 0.9938, 0.9689],
        [0.9956, 0.9987, 0.9789],
        [0.9837, 0.9949, 0.9927],
        [0.8655, 0.8977, 0.9519],
        [0.9978, 0.9962, 0.9721],
        [0.9990, 0.9957, 0.9713],
        [0.9962, 0.9982, 0.9785]], grad_fn=<MulBackward0>)

#### model forward in wrap up

In [18]:
from src import Model

In [19]:
model = Model(enc, clf)

In [27]:
logits = model(sq_idx, label_idx, batch)
logits

tensor([[0.9989, 0.9953, 0.9700],
        [0.9995, 0.9949, 0.9702],
        [0.9918, 0.9990, 0.9851],
        [0.9989, 0.9961, 0.9719],
        [0.9960, 0.9976, 0.9778],
        [0.9999, 0.9932, 0.9682],
        [0.9999, 0.9933, 0.9682],
        [0.9967, 0.9981, 0.9764],
        [0.9947, 0.9974, 0.9762],
        [0.9120, 0.9426, 0.9823],
        [0.9992, 0.9954, 0.9708],
        [0.9999, 0.9938, 0.9689],
        [0.3565, 0.4209, 0.5570],
        [0.9996, 0.9945, 0.9696],
        [0.8929, 0.9203, 0.9651],
        [0.9981, 0.9969, 0.9731],
        [0.9998, 0.9935, 0.9684],
        [0.9999, 0.9938, 0.9689],
        [0.9956, 0.9987, 0.9789],
        [0.9837, 0.9949, 0.9927],
        [0.8655, 0.8977, 0.9519],
        [0.9978, 0.9962, 0.9721],
        [0.9990, 0.9957, 0.9713],
        [0.9962, 0.9982, 0.9785]], grad_fn=<MulBackward0>)

In [24]:
loss = nn.CrossEntropyLoss()

In [26]:
label_idx[sq_idx == 1]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2])

In [28]:
loss(logits, label_idx[sq_idx == 1])

tensor(1.0978, grad_fn=<NllLossBackward0>)

In [31]:
logits.size()[0]

24

In [32]:
_, pred = logits.topk(1, 1, True, True)

In [38]:
pred.view(-1)

tensor([0, 0, 1, 0, 1, 0, 0, 1, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 1, 1, 2, 0, 0, 1])

In [39]:
correct = pred.view(-1).eq(label_idx[sq_idx == 1])
correct 

tensor([ True,  True, False,  True, False,  True,  True, False,  True, False,
        False, False, False, False, False, False, False, False, False, False,
         True, False, False, False])

In [49]:
correct.float().mean().item()

0.2916666567325592

In [50]:
def accuracy(logits, labels, topk=(1,)):
  with torch.no_grad():
    batch_size = logits.size(0)
    assert batch_size == labels.size(0)

    _, pred = logits.topk(max(topk), 1, True, True)
    pred = pred.view(-1)
    correct = pred.eq(labels)

    
    return correct.float().mean().item()

In [51]:
accuracy(logits,label_idx[sq_idx == 1] )

0.2916666567325592